# Topics


In [9]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time

In [2]:
sys.setrecursionlimit(1500)

In [7]:
with open("01-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'57643   17620\n19062   47340\n11105   16109\n72032   30050\n16289   65967\n42361   35795\n45873   16124\n16167   65832\n57216   47340\n15920   71163\n22534   30837\n38721   64603\n27502   32313\n17094   50986\n84849   67526\n20884   64392\n21987   59516\n39419   98547\n10839   29910\n77870   15917\n94474   57604\n29876   18768\n72281   16109\n52066   21561\n11847   24044\n83711   11404\n47340   29941\n54869   62664\n63495   62664\n87421   46184\n12673   51492\n21401   77546\n13800   70131\n35534   46986\n51304   41199\n51515   39683\n25160   55295\n39133   88509\n68329   62664\n56917   84849\n68841   33033\n27693   78397\n82953   22601\n47353   27226\n70002   59980\n74154   16109\n86034   84849\n95639   62349\n45722   59516\n69049   22317\n51692   17620\n39166   65967\n30495   55388\n81255   17869\n60454   41199\n26224   81511\n74983   16124\n55752   90759\n22465   24501\n57854   22943\n94700   59516\n83099   31049\n26617   32773\n23277   52176\n34125   18754\n61685   11874\n49752   4

In [5]:
test_data_raw = r"""3   4
4   3
2   5
1   3
3   9
3   3"""


def preprocess_data (data):
    # dtype='U10'
    left, right = [], []
    for row in data.split("\n"):
        left.append(int(row.split(" ")[0]))
        right.append(int(row.split(" ")[-1]))

    return left, right

test_data = preprocess_data(test_data_raw)
display(test_data)

([3, 4, 2, 1, 3, 3], [4, 3, 5, 3, 9, 3])

In [8]:
data = preprocess_data(data_raw)
data

([57643,
  19062,
  11105,
  72032,
  16289,
  42361,
  45873,
  16167,
  57216,
  15920,
  22534,
  38721,
  27502,
  17094,
  84849,
  20884,
  21987,
  39419,
  10839,
  77870,
  94474,
  29876,
  72281,
  52066,
  11847,
  83711,
  47340,
  54869,
  63495,
  87421,
  12673,
  21401,
  13800,
  35534,
  51304,
  51515,
  25160,
  39133,
  68329,
  56917,
  68841,
  27693,
  82953,
  47353,
  70002,
  74154,
  86034,
  95639,
  45722,
  69049,
  51692,
  39166,
  30495,
  81255,
  60454,
  26224,
  74983,
  55752,
  22465,
  57854,
  94700,
  83099,
  26617,
  23277,
  34125,
  61685,
  49752,
  82839,
  28748,
  19753,
  76703,
  41076,
  94607,
  88668,
  56446,
  93070,
  25427,
  51967,
  23531,
  44724,
  32642,
  54203,
  62977,
  31242,
  94016,
  55953,
  35527,
  54207,
  44027,
  82186,
  72150,
  68617,
  57085,
  18164,
  28818,
  94464,
  73344,
  76429,
  25559,
  24143,
  12888,
  33654,
  13309,
  90824,
  42199,
  99525,
  60919,
  14832,
  83140,
  68116,
  35265,
 

In [12]:
def solution (data, verbose=False):

    left_sorted = sorted(data[0])
    right_sorted = sorted(data[1])


    return sum( [abs(a-b) for a,b in zip(left_sorted, right_sorted)])



sol = solution(data)
# sol = solution(test_data, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))


2086478


# Part 2

In [20]:
def solution2 (data, verbose=False):
    
    counter = Counter()
    counter.update(data[1])
    return sum([key* counter[key] for key in data[0]])
    
    
sol = solution2(data)
# sol = solution2(test_data, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))   
    

24941624
